In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error

from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

2022-08-02 11:04:09.092241: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-02 11:04:09.092265: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
input_df = pd.read_csv("data/input_data_nn.csv", sep=";")
output_df = pd.read_csv("data/output_data_nn.csv", sep=";")

In [27]:
df = pd.concat([input_df, output_df], axis=1)

In [12]:
cols = output_df.columns.tolist() + input_df.columns.tolist()

In [26]:
cols = [
 'pot_reativa_inj_barramento11',
# 'pot_reativa_inj_barramento12',
# 'pot_reativa_inj_barramento24',
 'pot_ativa_inj_barramento11',
# 'pot_ativa_inj_barramento12',
# 'pot_ativa_inj_barramento24',    
 'tensao_barramento11',
# 'tensao_barramento12',
#  'tensao_barramento24',
  'carga_subrede_138kv',
  'pot_eolica_subrede_138kv',
  'pot_solar_subrede_138kv',
#  'status_1_gerador_subrede_230kv',
#  'status_2_gerador_subrede_230kv',
#  'status_3_gerador_subrede_230kv',
#  'status_4_gerador_subrede_230kv',
#  'status_5_gerador_subrede_230kv',
#  'status_6_gerador_subrede_230kv',
#  'status_7_gerador_subrede_230kv',
#  'status_8_gerador_subrede_230kv',
#  'status_9_gerador_subrede_230kv',
#  'status_10_gerador_subrede_230kv',
#  'status_11_gerador_subrede_230kv',
#  'status_12_gerador_subrede_230kv',
#  'status_13_gerador_subrede_230kv',
#  'status_14_gerador_subrede_230kv',
#  'status_15_gerador_subrede_230kv',
#  'status_16_gerador_subrede_230kv',
#  'status_17_gerador_subrede_230kv',
#  'status_18_gerador_subrede_230kv',
#  'status_1_linha_subrede_230kv',
#  'status_2_linha_subrede_230kv',
#  'status_3_linha_subrede_230kv',
#  'status_4_linha_subrede_230kv',
#  'status_5_linha_subrede_230kv',
#  'status_6_linha_subrede_230kv',
#  'status_7_linha_subrede_230kv',
#  'status_8_linha_subrede_230kv',
#  'status_9_linha_subrede_230kv',
#  'status_10_linha_subrede_230kv',
#  'status_11_linha_subrede_230kv',
#  'status_12_linha_subrede_230kv',
#  'status_13_linha_subrede_230kv',
#  'status_14_linha_subrede_230kv',
#  'status_15_linha_subrede_230kv',
#  'status_16_linha_subrede_230kv',
#  'status_17_linha_subrede_230kv',
#  'status_18_linha_subrede_230kv',
#  'status_19_linha_subrede_230kv',
#  'status_20_linha_subrede_230kv',
#  'status_21_linha_subrede_230kv'
]

In [28]:
df = df[cols]

In [29]:
test_split=round(len(df)*0.30)
df_for_training=df[:-test_split]
df_for_testing=df[-test_split:]
print(df_for_training.shape)
print(df_for_testing.shape)

(6132, 6)
(2628, 6)


In [30]:
scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled=scaler.transform(df_for_testing)

array([[4.83165917e-01, 2.23414479e-01, 8.85093517e-01, 5.55819477e-01,
        6.32287509e-01, 0.00000000e+00],
       [6.90066994e-01, 1.40612011e-02, 5.79535979e-01, 2.89786223e-01,
        6.33417710e-01, 0.00000000e+00],
       [6.77965974e-01, 5.20553071e-02, 5.93556780e-01, 2.61282660e-01,
        5.70590233e-01, 0.00000000e+00],
       ...,
       [5.72407611e-01, 1.80782404e-01, 5.90426301e-01, 5.79572446e-01,
        7.65566568e-01, 3.37452422e-01],
       [3.82452734e-01, 1.69935018e-01, 8.04570513e-06, 6.74584323e-01,
        8.99627950e-01, 3.11403605e-01],
       [3.79769801e-01, 1.71499171e-01, 8.02700704e-06, 6.79334916e-01,
        9.01468828e-01, 3.43657936e-01]])

In [34]:
days = 30 * 1
max_idx_pred = 1

In [5]:
model = keras.models.load_model('best_model.h5')

2022-08-02 11:04:12.253914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-02 11:04:12.253958: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-02 11:04:12.253993: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schlickmann): /proc/driver/nvidia/version does not exist
2022-08-02 11:04:12.255131: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [35]:
def createXY(dataset, max_idx_pred, n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i, 0:max_idx_pred])
    return np.array(dataX),np.array(dataY)

trainX,trainY=createXY(df_for_training_scaled, max_idx_pred, 24 * days)
testX,testY=createXY(df_for_testing_scaled, max_idx_pred, 24 * days)

print("trainX Shape-- ",trainX.shape)
print("trainY Shape-- ",trainY.shape)
print("testX Shape-- ",testX.shape)
print("testY Shape-- ",testY.shape)

trainX Shape--  (5412, 720, 6)
trainY Shape--  (5412, 1)
testX Shape--  (1908, 720, 6)
testY Shape--  (1908, 1)


In [ ]:
def build_model(callbacks):    
    grid_model = Sequential()
    grid_model.add(LSTM(50, return_sequences = True, input_shape=(24 * days, trainX.shape[-1])))
    grid_model.add(LSTM(50))
    grid_model.add(Dropout(0.2))
    #grid_model.add(LSTM(70, return_sequences = True))
    #grid_model.add(Dropout(0.2))
    #grid_model.add(LSTM(50))
    #grid_model.add(Dropout(0.2))
    grid_model.add(Dense(max_idx_pred))

    grid_model.compile(loss = 'mae',optimizer = 'adam')
    grid_model.fit(trainX,trainY, batch_size = 16, epochs = 50, callbacks=callbacks, validation_data = (testX, testY))
    
    return grid_model

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
mc = ModelCheckpoint('mae_50_15_1.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

model = build_model([es, mc])

prediction=model.predict(testX)

Epoch 1/50
339/339 [==============================] - ETA: 0s - loss: 0.1100
Epoch 1: val_loss improved from inf to 0.09913, saving model to mae_50_15_1.h5
339/339 [==============================] - 202s 589ms/step - loss: 0.1100 - val_loss: 0.0991
Epoch 2/50
339/339 [==============================] - ETA: 0s - loss: 0.0884
Epoch 2: val_loss improved from 0.09913 to 0.08362, saving model to mae_50_15_1.h5
339/339 [==============================] - 191s 563ms/step - loss: 0.0884 - val_loss: 0.0836
Epoch 3/50
339/339 [==============================] - ETA: 0s - loss: 0.0783
Epoch 3: val_loss improved from 0.08362 to 0.07852, saving model to mae_50_15_1.h5
339/339 [==============================] - 201s 594ms/step - loss: 0.0783 - val_loss: 0.0785
Epoch 4/50
 29/339 [=>............................] - ETA: 3:00 - loss: 0.0756

In [23]:
def rmse(index):
    plt.plot(originals[:,index], color = 'red', label = 'Real')
    plt.plot(pred[:,index], color = 'blue', label = 'Predicted')
    plt.legend()
    plt.show()
    return sqrt(mean_squared_error(originals[:,index], pred[:,index]))

In [20]:
def inverse_transform(arrs):
    zeros_padding = np.zeros(testX.shape[-1] - (max_idx_pred))
    arrs_copies = list(map(lambda arr: np.concatenate((arr, zeros_padding)), arrs))
    return scaler.inverse_transform(np.reshape(arrs_copies, (len(arrs), testX.shape[-1])))

In [115]:
pred = inverse_transform(prediction)
originals = inverse_transform(testY)

In [166]:
print("pred 0: " + str(pred[:,0]))
print("original 0: " + str(originals[:,0]))
print()
print("pred 1: " + str(pred[:,1]))
print("original 1: " + str(originals[:,1]))
print()
print("pred 2: " + str(pred[:,2]))
print("original 2: " + str(originals[:,2]))
print()
print("pred 3: " + str(pred[:,2]))
print("original 3: " + str(originals[:,2]))
print()
print("pred 4: " + str(pred[:,3]))
print("original 4: " + str(originals[:,3]))
print()

pred tensao_barramento11: [1.01334414 0.99277805 0.99481454 ... 1.01292363 0.9870933  0.94952426]
original tensao_barramento11: [1.00269358 1.00453829 1.00513947 ... 1.0019635  0.95127084 0.95126964]

pred tensao_barramento12: [1.01235144 0.99268041 0.99489139 ... 1.01326569 0.98885    0.94981958]
original tensao_barramento12: [0.99990073 1.00272539 1.00364593 ... 1.00335917 0.95157989 0.95158083]

pred tensao_barramento24: [1.00026255 0.97675341 0.97980152 ... 1.00912616 0.98568991 0.9514155 ]
original tensao_barramento24: [0.98334365 0.98592246 0.98566991 ... 1.00685737 0.95000062 0.95000064]

pred tensao_barramento24: [1.00026255 0.97675341 0.97980152 ... 1.00912616 0.98568991 0.9514155 ]
original tensao_barramento24: [0.98334365 0.98592246 0.98566991 ... 1.00685737 0.95000062 0.95000064]

pred pot_ativa_inj_barramento11: [77.61293781 60.00705901 49.46027855 ... 85.50988677 61.05178586
 53.62100092]
original pot_ativa_inj_barramento11: [66.24799586 54.39864737 48.00403871 ... 72.603

## Grid search

In [163]:
def build_model(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(50,return_sequences=True,input_shape=(30,51)))
    grid_model.add(LSTM(50))
    grid_model.add(Dropout(0.2))
    grid_model.add(Dense(1))

    grid_model.compile(loss = 'mse',optimizer = optimizer)
    # {'batch_size': 16, 'epochs': 10, 'optimizer': 'adam'}
    grid_model.fit(trainX,trainY, batch_size =  16, epochs = 10,validation_data=(testX, testY))
    
    return grid_model

grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(testX,testY))

parameters = {'batch_size' : [16,20],
              'epochs' : [8,10],
              'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            cv = 2)


grid_search = grid_search.fit(trainX,trainY)
grid_search.best_params_
my_model=grid_search.best_estimator_.model

/tmp/ipykernel_10384/1044546172.py:13: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(testX,testY))


In [305]:
prediction_copies_array = np.repeat(prediction, 25, axis=1)
prediction_copies_array_m = list(map(lambda pred: (pred.tolist() * 25) + [pred.tolist()[0]], prediction))
pred = scaler.inverse_transform(np.reshape(prediction_copies_array_m,(len(prediction),51)))[:,1]

In [35]:
def build_model_cv(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(50,return_sequences=True,input_shape=(24 * days, trainX.shape[-1])))
    #grid_model.add(LSTM(50))
    grid_model.add(Dropout(0.2))
    grid_model.add(Dense(max_idx_pred))

    grid_model.compile(loss = 'mse',optimizer = optimizer)
    
    return grid_model

grid_model = KerasRegressor(build_fn=build_model_cv,verbose=1,validation_data=(testX,testY))

parameters = {'batch_size' : [16,20],
              'epochs' : [8,10],
              'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            cv = 2)

grid_search = grid_search.fit(trainX,trainY)
grid_search.best_params_
my_model=grid_search.best_estimator_.model

/tmp/ipykernel_7967/479307926.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_model = KerasRegressor(build_fn=build_model_cv,verbose=1,validation_data=(testX,testY))


Epoch 1/8
188/188 [==============================] - 25s 123ms/step - loss: 0.0305 - val_loss: 0.0203
Epoch 2/8
188/188 [==============================] - 24s 130ms/step - loss: 0.0139 - val_loss: 0.0158
Epoch 3/8
188/188 [==============================] - 24s 130ms/step - loss: 0.0107 - val_loss: 0.0110
Epoch 4/8
188/188 [==============================] - 23s 122ms/step - loss: 0.0087 - val_loss: 0.0106
Epoch 5/8
188/188 [==============================] - 22s 119ms/step - loss: 0.0078 - val_loss: 0.0091
Epoch 6/8
188/188 [==============================] - 23s 121ms/step - loss: 0.0068 - val_loss: 0.0071
Epoch 7/8
188/188 [==============================] - 23s 123ms/step - loss: 0.0066 - val_loss: 0.0079
Epoch 8/8
188/188 [==============================] - 6s 34ms/step - loss: 0.0076
Epoch 1/8
188/188 [==============================] - 25s 119ms/step - loss: 0.0267 - val_loss: 0.0194
Epoch 2/8
188/188 [==============================] - 23s 122ms/step - loss: 0.0152 - val_loss: 0.0132
E

Epoch 6/10
376/376 [==============================] - 23s 61ms/step - loss: 0.0060 - val_loss: 0.0051
Epoch 7/10
376/376 [==============================] - 23s 62ms/step - loss: 0.0056 - val_loss: 0.0052
Epoch 8/10
376/376 [==============================] - 23s 62ms/step - loss: 0.0054 - val_loss: 0.0054
Epoch 9/10
376/376 [==============================] - 23s 62ms/step - loss: 0.0053 - val_loss: 0.0047
Epoch 10/10
376/376 [==============================] - 23s 62ms/step - loss: 0.0051 - val_loss: 0.0046


In [36]:
grid_search.best_params_

{'batch_size': 16, 'epochs': 10, 'optimizer': 'adam'}